# Convergence of Linear Multistep Methods

This notebook shows three methods applied to the same problem $u\prime(t) = \lambda (u(t)-\cos(t)) - \sin(t)$ with data $u(0) = \eta = 1$, for which the true solution is just $u(t) = \cos(t)$.

Note the following:
- The Forward Euler method looks bad if $k$ is too large but for small enough $k$ looks fine, and converges.
- The midpoint method converges up to any finite $T$ as the timestep is reduced, but eventually goes bad no matter how small $k$ is.  It is zero-stable but we will see there are issues with "absolute stability".
- The third method is non-convergent for any $T>0$ even though it is consistent and the local truncation error is $O(k)$.
- The value $\lambda =-5$ is used here.  Changing this value doesn't affect the true solution (when $\eta = 1$) but does affect the behavior of the numerical methods.

In [ ]:
%pylab inline

In [ ]:
from ipywidgets import interact

In [ ]:
lam = -5.
t0 = 0.
tfinal = 10.
eta = 1.

# function f(u,t) (as a "lambda function")
f = lambda u,t: lam*(u-cos(t)) - sin(t)

utrue = lambda t: exp(lam*(t-t0))*(eta - cos(t0)) + cos(t)
tfine = linspace(t0, tfinal, 1000)  # fine grid for plotting true solution
ufine = utrue(tfine)
plot(tfine,ufine)
title('True solution')

## Forward Euler

In [ ]:
def euler(nsteps):
    t = linspace(t0, tfinal, nsteps+1)
    dt = t[1] - t[0]
    U = empty(nsteps+1)  # array for computed solution
    U[0] = eta
    for n in range(nsteps):
        U[n+1] = U[n] + dt * f(U[n], t[n])
    figure(figsize=(8,4))
    plot(t,U,'r-o', label='Euler')
    plot(tfine, ufine, 'b', label='true')
    xlim(0,12)
    legend()
    title('%i steps, dt = %7.4f' % (nsteps, dt))
    

In [ ]:
euler(25)

You can change the value of `nsteps` in the cell above and re-execute, or we can use an interactive widget to display results for all values of `nsteps` between 20 and 30:

In [ ]:
interact(euler, nsteps=(20,30,1))

## Midpoint method

$U^{n+1} = U^{n-1} + 2k f(U^n, t_n)$.

We will use forward Euler to compute $U^1$ from $U^0=\eta$.

In [ ]:
def midpoint(nsteps):
    t = linspace(t0, tfinal, nsteps+1)
    dt = t[1] - t[0]
    U = empty(nsteps+1)  # array for computed solution
    U[0] = eta
    U[1] = eta + dt * f(U[0], t[0])  # Forward Euler 
    for n in range(1,nsteps):
        U[n+1] = U[n-1] + 2*dt * f(U[n], t[n])
        if abs(U[n+1]) > 10:
            break # since it grows exponentially
    figure(figsize=(8,4))
    plot(t[:n+1],U[:n+1],'r-o', label='Midpoint')
    plot(tfine, ufine, 'b', label='true')
    xlim(0,12)
    legend()
    title('%i steps, dt = %7.4f' % (nsteps, dt))
    

In [ ]:
midpoint(50)

## A non-convergent method

Now consider the LMM 
$$
U^{n+2} - 3U^{n+1} + 2U^n = -k f(U^n,t_n).
$$
The local truncation error is $\tau^n = \frac 5 2 k u\prime\prime(t_n) + O(k^2)$, so it is consistent and locally "first-order accurate".  But it does not converge at any fixed $T>0$. 

In [ ]:
def bad_method(nsteps):
    t = linspace(t0, tfinal, nsteps+1)
    dt = t[1] - t[0]
    U = empty(nsteps+1)  # array for computed solution
    U[0] = eta
    U[1] = eta + dt * f(U[0], t[0])  # Forward Euler 
    for n in range(1,nsteps):
        U[n+1] = 3*U[n] - 2*U[n-1] - dt * f(U[n], t[n])
        if abs(U[n+1]) > 10:
            break # since it grows exponentially
    figure(figsize=(8,4))
    plot(t[:n+1],U[:n+1],'r-o', label='Bad method')
    plot(tfine, ufine, 'b', label='true')
    xlim(0,12)
    legend()
    title('%i steps, dt = %7.4f' % (nsteps, dt))

In [ ]:
bad_method(50)